In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from nameparser import HumanName
import requests

In [2]:
team = 'SD'
df = pd.DataFrame()

# 2018 - 2020


In [3]:
_source = {
    #2020:
}

In [4]:
employees = {
    2020: {
        
    }
}

# 2021 - Present

In [5]:
_source = {
    2023: 'https://web.archive.org/web/20230326092234/https://www.mlb.com/padres/team/front-office',
    2022: 'https://web.archive.org/web/20220323041148/https://www.mlb.com/padres/team/front-office'
    # 2021: 'https://web.archive.org/web/20210331155729/https://www.mlb.com/padres/team/front-office'
}

In [6]:
df = pd.DataFrame()

for year in _source.keys():
    res = requests.get(_source[year])
    soup = BeautifulSoup(res.content)

    raw = [y for y in [x.text.strip('\n').split('\n') for x in soup.select('h3, div.p-table__title, tr')] if y[0] != 'Title']
    deps = [x.text for x in soup.select('h3.p-heading__text')]

    department = None
    office = None

    for _ in raw:
        if _[0] in deps:
            department = _[0].title()
            office = None
        elif len(_) == 1:
            office = _[0].title()
        else:
            title = _[0]
            name = _[1]
            
            df = pd.concat([
                df, 
                pd.DataFrame({
                    'Year': year,
                    'Team': team,
                    'NameLast': [HumanName(name).last],
                    'NameFirst': [HumanName(name).first],
                    'Title': [title],
                    'Department': [department],
                    'Source of Information': [_source[year]]
                })])

df


,Year,Team,NameLast,NameFirst,Title,Department,Source of Information
0,2023,SD,Seidler,Peter,Chairman,Executives,https://web.archive.org/web/20230326092234/htt...
0,2023,SD,Greupner,Erik,Chief Executive Officer,Executives,https://web.archive.org/web/20230326092234/htt...
0,2023,SD,Preller,A.J.,President of Baseball Operations & General Man...,Executives,https://web.archive.org/web/20230326092234/htt...
0,2023,SD,Perry,Caroline,Chief Operating Officer,Executives,https://web.archive.org/web/20230326092234/htt...
0,2023,SD,Schang,Virginia,Executive Assistant to the CEO,Executive Support Staff,https://web.archive.org/web/20230326092234/htt...
...,...,...,...,...,...,...,...
0,2022,SD,Lugo,Jessica,"Account Specialist, Membership Services",Ticket Sales,https://web.archive.org/web/20220323041148/htt...
0,2022,SD,West,Jason,"Account Specialist, Membership Services",Ticket Sales,https://web.archive.org/web/20220323041148/htt...
0,2022,SD,Sanchez,Crystal,"Account Specialist, Membership Services",Ticket Sales,https://web.archive.org/web/20220323041148/htt...
0,2022,SD,Dill,Chelsea,"Director, Tourism & Corporate Events",Ticket Sales,https://web.archive.org/web/20220323041148/htt...


In [8]:
df[['Year', 'Team', 'NameLast', 'NameFirst', 'Title', 'Department', 'Source of Information']] \
    .to_excel(f'../out/{team}-{min(_source.keys())}-to-{max(_source.keys())}.xlsx', index=False)